In [2]:
import wikipediaapi
import os
import pandas as pd
import sys
import spacy

In [127]:
def get_page_summaries(pages, lang, look_for_other_lang=False, other_lang='de'):

    summaries = []
    wiki = wikipediaapi.Wikipedia(lang)
    
    for page in pages:
        try:
            page_en = wiki.page(page)
            if look_for_other_lang:
                page_other = page_en.langlinks.get(other_lang, None)
                page_other_title = page_other.title if page_other else None
                page_other_summary = page_other.summary if page_other else None
                summaries.append({'article_en': page, 'summary_en': page_en.summary, 
                          f'article_{other_lang}': page_other_title, f'summary_{other_lang}': page_other_summary})
            else:
                summaries.append({'article_en': page, 'summary_en': page_en.summary})

        except:
            print(f"problem fetching data for page: {page}", file=sys.stderr)
            continue
        
    return pd.DataFrame(summaries)

# replaced attack by offensivem, nuclear by nuclear weapon, terror by Terror (politics), agro by agro-terrorism, In order to get meaningful results


In [128]:
terrorism_articles = ['Abu Sayyaf', 'afghanistan', 'Agro-terrorism', 'al-qaeda',
       'Al-Qaeda in the Arabian Peninsula',
       'Al-Qaeda in the Islamic Maghreb', 'al-shabaab',
       'ammonium_nitrate', 'Offensive (military)', 'biological_weapon', 'car_bomb',
       'chemical_weapon', 'conventional_weapon', 'dirty_bomb',
       'eco-terrorism', 'environmental_terrorism',
       'Euskadi ta Askatasuna', 'extremism', 'farc', 'fundamentalism',
       'hamas', 'hezbollah', 'improvised_explosive_device', 'iran',
       'iraq', 'Irish Republican Army', 'islamist', 'jihad',
       'nationalism', 'nigeria', 'Nuclear weapon', 'nuclear_enrichment',
       'pakistan', 'Palestine Liberation Front', 'pirates', 'PLO',
       'political_radicalism', 'recruitment', 'somalia', 'suicide_attack',
       'suicide_bomber', 'taliban', 'tamil_tigers',
       'Tehrik-i-Taliban Pakistan', 'Terror (politics)', 'terrorism',
       'weapons-grade', 'yemen']

In [129]:
summaries = get_page_summaries(terrorism_articles, 'en', look_for_other_lang=True)

In [130]:
summaries.head()

,article_en,summary_en,article_de,summary_de
0,Abu Sayyaf,Abu Sayyaf ( (listen); Arabic: جماعة أبو سياف‎...,Abu Sajaf,"Abu Sayyaf (أبو سيّاف, DMG Abū sayyāf ‚Schwert..."
1,afghanistan,"Afghanistan ( (listen), also pronounced /ævˈɡæ...",Afghanistan,Afghanistan (paschtunisch und persisch افغانست...
2,Agro-terrorism,"Agroterrorism, also known as agriterrorism and...",Agroterrorismus,Agroterrorismus (engl. agro-terrorism oder agr...
3,al-qaeda,"Al-Qaeda (; Arabic: القاعدة‎ al-Qāʿidah, IPA: ...",Al-Qaida,"Al-Qaida, auch al-Kaida (arabisch القاعدة, DMG..."
4,Al-Qaeda in the Arabian Peninsula,Al-Qaeda in the Arabian Peninsula (Arabic: تنظ...,Al-Qaida auf der Arabischen Halbinsel,Al-Qaida auf der arabischen Halbinsel (arabisc...


In [ ]:
def get_members_of_category(cat, lang, level=0, max_level=1):
    wiki = wikipediaapi.Wikipedia(lang)
    category_page = wiki.page(cat)
    all_pages_in_cat = []
    for c in category_page.categorymembers.values(): 
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            all_pages_in_cat.extend(get_members_of_category(c, lang, level+1, max_level))
        else: 
            all_pages_in_cat.append(c.title)
    
    return all_pages_in_cat

In [108]:
categories_selected = ['Kategorie:Afghanistan', 'Kategorie:Afghanistankrieg (seit 1978)', 'Kategorie:Al-Qaida',
                       'Kategorie:Antisemitismus', 'Kategorie:Anti-Freimaurerei','Kategorie:Antizionismus',
                       'Kategorie:Asymmetrische Kriegführung', 'Kategorie:Attentat', 'Kategorie:Bewaffnete islamistische Organisation',
                       'Kategorie:Bewaffnete islamistische Organisation', 'Kategorie:Bewaffnete islamistische Untergrundorganisation',
                       'Kategorie:Bombe', 'Kategorie:Hamas', 'Kategorie:Hisbollah','Kategorie:Islamische Republik',
                        'Kategorie:Islamistischer Terrorismus','Kategorie:Terrorismus', 'Kategorie:Taliban']

In [138]:
potential_pages = set(summaries['article_de'])
for category in categories_selected:
    potential_pages.update(get_members_of_category(category,'de', 0, 3))

In [140]:
summaries_de = get_page_summaries(potential_pages, 'de')

problem fetching data for page: None


In [144]:
summaries_de.dropna()

,article_en,summary_en
0,Liechtenstein in der Zeit des Nationalsozialismus,Liechtenstein in der Zeit des Nationalsozialis...
1,Zentrale Dienststelle für Juden,Die Zentrale Dienststelle für Juden (auch Zent...
2,Hamas-Charta,Die Hamas-Gründungscharta ist die Gründungsurk...
3,International Islamic Relief Organization,"Die International Islamic Relief Organization,..."
4,Operation Neptune Spear,Die Operation Neptune Spear (deutsch „Unterneh...
...,...,...
729,Payitaht Abdülhamid,"Payitaht Abdülhamid, im englischen Sprachraum ..."
730,Finanzoligarchie,Finanzoligarchie bezeichnet eine Oligarchie vo...
731,Regenschirmattentat,Als Regenschirmattentat wird der Mordanschlag ...
732,Ideologiekritik,Ideologiekritik bezeichnet ein philosophisches...


In [145]:
summaries_en = summaries[['article_en','summary_en']]

In [157]:
summaries_en.to_csv('summaries_en.csv')

In [158]:
summaries_de.to_csv('summaries_de.csv')

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
summaries_en.csv = pd.read_csv('data/summaries_en.csv', index_col=0)
summaries_en['summary_en'] =  summaries_en['summary_en'].str.replace(r'(\([^)]*\))| (\[[^\]]*\])', '')

NameError: name 'summaries_en' is not defined